# Registration of two images

## Loading the two images

In [2]:
import SimpleITK as sitk  
import os
dataset = './resources/'
filenames = os.listdir(dataset)
print(filenames)

['VF-MRT1-1014-1174.vtk', 'VF-MRT2-1014-1174.vtk']


In [3]:
import numpy as np
fixedFile = dataset + filenames[0]
fixed_image = sitk.ReadImage(fixedFile,sitk.sitkFloat32)
fixedImageArray = sitk.GetArrayFromImage(fixed_image)
movingFile = dataset + filenames[1]
moving_image = sitk.ReadImage(movingFile,sitk.sitkFloat32)
movingImageArray = sitk.GetArrayFromImage(moving_image)
# movingImageArray = movingImageArray[:fixedImageArray.shape[0],:,:]
# moving_image = sitk.GetImageFromArray(movingImageArray)
print(fixedImageArray.shape)
print(movingImageArray.shape)

(33, 256, 256)
(63, 256, 256)


## Visualizing the images

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt

from ipywidgets import interact, fixed
from IPython.display import clear_output

# Callback invoked by the interact IPython method for scrolling through the image stacks of
# the two images (moving and fixed).
def display_images(fixed_image_z, moving_image_z, fixed_npa, moving_npa):
    # Create a figure with two subplots and the specified size.
    plt.subplots(1,2,figsize=(10,8))
    
    # Draw the fixed image in the first subplot.
    plt.subplot(1,2,1)
    plt.imshow(fixed_npa[fixed_image_z,:,:],cmap=plt.cm.Greys_r)
    plt.title('fixed image')
    plt.axis('off')
    
    # Draw the moving image in the second subplot.
    plt.subplot(1,2,2)
    plt.imshow(moving_npa[moving_image_z,:,:],cmap=plt.cm.Greys_r)
    plt.title('moving image')
    plt.axis('off')
    
    plt.show()

# Callback invoked by the IPython interact method for scrolling and modifying the alpha blending
# of an image stack of two images that occupy the same physical space. 
def display_images_with_alpha(image_z, alpha, fixed, moving):
    img = (1.0 - alpha)*fixed[:,:,image_z] + alpha*moving[:,:,image_z] 
    plt.imshow(sitk.GetArrayViewFromImage(img),cmap=plt.cm.Greys_r)
    plt.axis('off')
    plt.show()
    
# Callback invoked when the StartEvent happens, sets up our new data.
def start_plot():
    global metric_values, multires_iterations
    
    metric_values = []
    multires_iterations = []

# Callback invoked when the EndEvent happens, do cleanup of data and figure.
def end_plot():
    global metric_values, multires_iterations
    
    del metric_values
    del multires_iterations
    # Close figure, we don't want to get a duplicate of the plot latter on.
    plt.close()

# Callback invoked when the IterationEvent happens, update our data and display new figure.
def plot_values(registration_method):
    global metric_values, multires_iterations
    
    metric_values.append(registration_method.GetMetricValue())                                       
    # Clear the output area (wait=True, to reduce flickering), and plot current data
    clear_output(wait=True)
    # Plot the similarity metric values
    plt.plot(metric_values, 'r')
    plt.plot(multires_iterations, [metric_values[index] for index in multires_iterations], 'b*')
    plt.xlabel('Iteration Number',fontsize=12)
    plt.ylabel('Metric Value',fontsize=12)
    plt.show()
    
# Callback invoked when the sitkMultiResolutionIterationEvent happens, update the index into the 
# metric_values list. 
def update_multires_iterations():
    global metric_values, multires_iterations
    multires_iterations.append(len(metric_values))

In [5]:
interact(display_images, fixed_image_z=(0,fixed_image.GetSize()[2]-1), moving_image_z=(0,moving_image.GetSize()[2]-1), fixed_npa = fixed(sitk.GetArrayViewFromImage(fixed_image)), moving_npa=fixed(sitk.GetArrayViewFromImage(moving_image)))

interactive(children=(IntSlider(value=16, description='fixed_image_z', max=32), IntSlider(value=31, descriptio…

<function __main__.display_images(fixed_image_z, moving_image_z, fixed_npa, moving_npa)>

Alpha blending is used to get transperancy of the added images  
Alpha = 0.5 shows both images 
It is clear that there is no complete overlap between the two images.  

In [6]:

interact(display_images_with_alpha, image_z=(0,fixed_image.GetSize()[2]-1), alpha=(0.0,1.0,0.05), fixed = fixed(fixed_image), moving=fixed(moving_image))

interactive(children=(IntSlider(value=16, description='image_z', max=32), FloatSlider(value=0.5, description='…

<function __main__.display_images_with_alpha(image_z, alpha, fixed, moving)>

## Setting the initial transform

In [46]:
# This is the registration configuration which we use in all cases. The only parameter that we vary 
# is the initial_transform. 
def multires_registration(fixed_image, moving_image, initial_transform):
    registration_method = sitk.ImageRegistrationMethod()
    registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
    registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
    registration_method.SetMetricSamplingPercentage(0.01)
    registration_method.SetInterpolator(sitk.sitkLinear)
    registration_method.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=100, estimateLearningRate=registration_method.Once)
    registration_method.SetOptimizerScalesFromPhysicalShift() 
    registration_method.SetInitialTransform(initial_transform, inPlace=False)
    registration_method.SetShrinkFactorsPerLevel(shrinkFactors = [4,2,1])
    registration_method.SetSmoothingSigmasPerLevel(smoothingSigmas = [2,1,0])
    registration_method.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()

    final_transform = registration_method.Execute(fixed_image, moving_image)
    print('Final metric value: {0}'.format(registration_method.GetMetricValue()))
    print('Optimizer\'s stopping condition, {0}'.format(registration_method.GetOptimizerStopConditionDescription()))
    return (final_transform, registration_method.GetMetricValue())

In [8]:
fixed_image1 = fixed_image
moving_image1 = moving_image
initial_transform = sitk.CenteredTransformInitializer(fixed_image1, 
                                                      moving_image1, 
                                                      sitk.Euler3DTransform(), 
                                                      sitk.CenteredTransformInitializerFilter.GEOMETRY)
registration_method = sitk.ImageRegistrationMethod()
registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
registration_method.SetMetricSamplingPercentage(0.01)
registration_method.SetInterpolator(sitk.sitkLinear)
# The order of parameters for the Euler3DTransform is [angle_x, angle_y, angle_z, t_x, t_y, t_z]. The parameter 
# sampling grid is centered on the initial_transform parameter values, that are all zero for the rotations. Given
# the number of steps and their length and optimizer scales we have:
# angle_x = 0
# angle_y = -pi, 0, pi
# angle_z = -pi, 0, pi
registration_method.SetOptimizerAsExhaustive(numberOfSteps=[0,1,1,0,0,0], stepLength = np.pi)
registration_method.SetOptimizerScales([1,1,1,1,1,1])

#Perform the registration in-place so that the initial_transform is modified.
registration_method.SetInitialTransform(initial_transform, inPlace=True)
registration_method.Execute(fixed_image1, moving_image1)

print('best initial transformation is: ' + str(initial_transform.GetParameters()))

best initial transformation is: (0.0, -3.141592653589793, -3.141592653589793, 0.0, 0.0, 75.0)


In [9]:
final_transform, _ = multires_registration(fixed_image1, moving_image1, initial_transform)
moving_resampled = sitk.Resample(moving_image1, fixed_image1, final_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())

interact(display_images_with_alpha, image_z=(0,fixed_image.GetSize()[2] - 1), alpha=(0.0,1.0,0.05), fixed = fixed(fixed_image1), moving=fixed(moving_resampled))

Final metric value: -0.35995963508929757
Optimizer's stopping condition, GradientDescentOptimizerv4Template: Convergence checker passed at iteration 29.


interactive(children=(IntSlider(value=16, description='image_z', max=32), FloatSlider(value=0.5, description='…

<function __main__.display_images_with_alpha(image_z, alpha, fixed, moving)>

In [10]:
fixed_image2 = fixed_image
moving_image2 = moving_image
initial_transform = sitk.CenteredTransformInitializer(fixed_image2, 
                                                      moving_image2, 
                                                      sitk.Euler3DTransform(), 
                                                      sitk.CenteredTransformInitializerFilter.MOMENTS)
registration_method = sitk.ImageRegistrationMethod()
registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
registration_method.SetMetricSamplingPercentage(0.01)
registration_method.SetInterpolator(sitk.sitkLinear)
# The order of parameters for the Euler3DTransform is [angle_x, angle_y, angle_z, t_x, t_y, t_z]. The parameter 
# sampling grid is centered on the initial_transform parameter values, that are all zero for the rotations. Given
# the number of steps and their length and optimizer scales we have:
# angle_x = 0
# angle_y = -pi, 0, pi
# angle_z = -pi, 0, pi
registration_method.SetOptimizerAsExhaustive(numberOfSteps=[0,1,1,0,0,0], stepLength = np.pi)
registration_method.SetOptimizerScales([1,1,1,1,1,1])

#Perform the registration in-place so that the initial_transform is modified.
registration_method.SetInitialTransform(initial_transform, inPlace=True)
registration_method.Execute(fixed_image2, moving_image2)

print('best initial transformation is: ' + str(initial_transform.GetParameters()))

best initial transformation is: (0.0, 0.0, 0.0, -0.8354386020256612, -11.594628496806024, 7.1451826604141075)


In [11]:
final_transform, _ = multires_registration(fixed_image2, moving_image2, initial_transform)
moving_resampled = sitk.Resample(moving_image2, fixed_image2, final_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())

interact(display_images_with_alpha, image_z=(0,fixed_image.GetSize()[2] - 1), alpha=(0.0,1.0,0.05), fixed = fixed(fixed_image2), moving=fixed(moving_resampled))

Final metric value: -0.7290967297054107
Optimizer's stopping condition, GradientDescentOptimizerv4Template: Convergence checker passed at iteration 9.


interactive(children=(IntSlider(value=16, description='image_z', max=32), FloatSlider(value=0.5, description='…

<function __main__.display_images_with_alpha(image_z, alpha, fixed, moving)>

In [12]:
def multires_registration_noInitial(fixed_image, moving_image):
    registration_method = sitk.ImageRegistrationMethod()
    registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
    registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
    registration_method.SetMetricSamplingPercentage(0.01)
    registration_method.SetInterpolator(sitk.sitkLinear)
    registration_method.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=100, estimateLearningRate=registration_method.Once)
    registration_method.SetOptimizerScalesFromPhysicalShift() 
    registration_method.SetShrinkFactorsPerLevel(shrinkFactors = [4,2,1])
    registration_method.SetSmoothingSigmasPerLevel(smoothingSigmas = [2,1,0])
    registration_method.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()

    final_transform = registration_method.Execute(fixed_image, moving_image)
    print('Final metric value: {0}'.format(registration_method.GetMetricValue()))
    print('Optimizer\'s stopping condition, {0}'.format(registration_method.GetOptimizerStopConditionDescription()))
    return (final_transform, registration_method.GetMetricValue())

In [15]:
fixed_image4 = fixed_image
moving_image4 = moving_image
final_transform, _ = multires_registration_noInitial(fixed_image4, moving_image4)
moving_resampled = sitk.Resample(moving_image4, fixed_image4, final_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())

interact(display_images_with_alpha, image_z=(0,fixed_image.GetSize()[2] - 1), alpha=(0.0,1.0,0.05), fixed = fixed(fixed_image4), moving=fixed(moving_resampled))

Final metric value: -0.4791522558373797
Optimizer's stopping condition, GradientDescentOptimizerv4Template: Convergence checker passed at iteration 31.


interactive(children=(IntSlider(value=16, description='image_z', max=32), FloatSlider(value=0.5, description='…

<function __main__.display_images_with_alpha(image_z, alpha, fixed, moving)>

In [34]:
fixed_image6 = fixed_image
moving_image6 = moving_image
initial_transform = sitk.CenteredTransformInitializer(fixed_image6, 
                                                      moving_image6, 
                                                      sitk.VersorRigid3DTransform())
registration_method = sitk.ImageRegistrationMethod()
registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
registration_method.SetMetricSamplingPercentage(0.01)
registration_method.SetInterpolator(sitk.sitkLinear)

registration_method.SetOptimizerAsExhaustive(numberOfSteps=[0,1,1,0,0,0], stepLength = np.pi)
registration_method.SetOptimizerScales([1,1,1,1,1,1])

#Perform the registration in-place so that the initial_transform is modified.
registration_method.SetInitialTransform(initial_transform, inPlace=True)
registration_method.Execute(fixed_image6, moving_image6)

print('best initial transformation is: ' + str(initial_transform.GetParameters()))

best initial transformation is: (0.0, 0.0, 0.0, -0.10125232367656167, -10.262984261211017, -0.9033173742011229)


In [35]:
final_transform, _ = multires_registration(fixed_image6, moving_image6, initial_transform)
moving_resampled = sitk.Resample(moving_image5, fixed_image6, final_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())

interact(display_images_with_alpha, image_z=(0,fixed_image.GetSize()[2] - 1), alpha=(0.0,1.0,0.05), fixed = fixed(fixed_image6), moving=fixed(moving_resampled))

Final metric value: -0.6832567490554737
Optimizer's stopping condition, GradientDescentOptimizerv4Template: Convergence checker passed at iteration 10.


interactive(children=(IntSlider(value=16, description='image_z', max=32), FloatSlider(value=0.5, description='…

<function __main__.display_images_with_alpha(image_z, alpha, fixed, moving)>

In [43]:
fixed_image7= fixed_image
moving_image7 = moving_image
transformDomainMeshSize = [8] * moving_image7.GetDimension()
initial_transform = sitk.TranslationTransform(fixed_image7.GetDimension())
registration_method = sitk.ImageRegistrationMethod()
registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
registration_method.SetMetricSamplingPercentage(0.01)
registration_method.SetInterpolator(sitk.sitkLinear)

registration_method.SetOptimizerAsExhaustive(numberOfSteps=[0,1,1,0,0,0], stepLength = np.pi)
registration_method.SetOptimizerScales([1,1,1,1,1,1])

#Perform the registration in-place so that the initial_transform is modified.
registration_method.SetInitialTransform(initial_transform, inPlace=True)
registration_method.Execute(fixed_image7, moving_image7)

print('best initial transformation is: ' + str(initial_transform.GetParameters()))

best initial transformation is: (0.0, -3.141592653589793, -3.141592653589793)


In [47]:
final_transform, _ = multires_registration(fixed_image7, moving_image7, initial_transform)
moving_resampled = sitk.Resample(moving_image7, fixed_image7, final_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())

interact(display_images_with_alpha, image_z=(0,fixed_image.GetSize()[2] - 1), alpha=(0.0,1.0,0.05), fixed = fixed(fixed_image7), moving=fixed(moving_resampled))

Final metric value: -0.6827188261937738
Optimizer's stopping condition, GradientDescentOptimizerv4Template: Convergence checker passed at iteration 10.


interactive(children=(IntSlider(value=16, description='image_z', max=32), FloatSlider(value=0.5, description='…

<function __main__.display_images_with_alpha(image_z, alpha, fixed, moving)>

The best is the Euler3DTransform with the Geomoetry setting
## Testing the interpolator

In [48]:
def multires_registration_interpolator(fixed_image, moving_image, initial_transform,interpolator):
    registration_method = sitk.ImageRegistrationMethod()
    registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
    registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
    registration_method.SetMetricSamplingPercentage(0.01)
    registration_method.SetInterpolator(interpolator)
    registration_method.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=100, estimateLearningRate=registration_method.Once)
    registration_method.SetOptimizerScalesFromPhysicalShift() 
    registration_method.SetInitialTransform(initial_transform, inPlace=False)
    registration_method.SetShrinkFactorsPerLevel(shrinkFactors = [4,2,1])
    registration_method.SetSmoothingSigmasPerLevel(smoothingSigmas = [2,1,0])
    registration_method.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()

    final_transform = registration_method.Execute(fixed_image, moving_image)
    print('Final metric value: {0}'.format(registration_method.GetMetricValue()))
    print('Optimizer\'s stopping condition, {0}'.format(registration_method.GetOptimizerStopConditionDescription()))
    return (final_transform, registration_method.GetMetricValue())

In [50]:
fixed_image1 = fixed_image
moving_image1 = moving_image
initial_transform = sitk.CenteredTransformInitializer(fixed_image1, 
                                                      moving_image1, 
                                                      sitk.Euler3DTransform(), 
                                                      sitk.CenteredTransformInitializerFilter.MOMENTS)
interpolator = sitk.sitkNearestNeighbor
final_transform, _ = multires_registration_interpolator(fixed_image1, moving_image1, initial_transform,interpolator)
moving_resampled = sitk.Resample(moving_image1, fixed_image1, final_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())

interact(display_images_with_alpha, image_z=(0,fixed_image.GetSize()[2] - 1), alpha=(0.0,1.0,0.05), fixed = fixed(fixed_image1), moving=fixed(moving_resampled))

Final metric value: -0.05969163223784532
Optimizer's stopping condition, GradientDescentOptimizerv4Template: Convergence checker passed at iteration 9.


interactive(children=(IntSlider(value=16, description='image_z', max=32), FloatSlider(value=0.5, description='…

<function __main__.display_images_with_alpha(image_z, alpha, fixed, moving)>

In [51]:
fixed_image2 = fixed_image
moving_image2 = moving_image
initial_transform = sitk.CenteredTransformInitializer(fixed_image2, 
                                                      moving_image2, 
                                                      sitk.Euler3DTransform(), 
                                                      sitk.CenteredTransformInitializerFilter.MOMENTS)
interpolator = sitk.sitkGaussian 
final_transform, _ = multires_registration_interpolator(fixed_image2, moving_image2, initial_transform,interpolator)
moving_resampled = sitk.Resample(moving_image2, fixed_image2, final_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())

interact(display_images_with_alpha, image_z=(0,fixed_image.GetSize()[2] - 1), alpha=(0.0,1.0,0.05), fixed = fixed(fixed_image2), moving=fixed(moving_resampled))

Final metric value: -0.6838248084818785
Optimizer's stopping condition, GradientDescentOptimizerv4Template: Convergence checker passed at iteration 11.


interactive(children=(IntSlider(value=16, description='image_z', max=32), FloatSlider(value=0.5, description='…

<function __main__.display_images_with_alpha(image_z, alpha, fixed, moving)>

In [52]:
fixed_image2 = fixed_image
moving_image2 = moving_image
initial_transform = sitk.CenteredTransformInitializer(fixed_image2, 
                                                      moving_image2, 
                                                      sitk.Euler3DTransform(), 
                                                      sitk.CenteredTransformInitializerFilter.MOMENTS)
interpolator = sitk.sitkHammingWindowedSinc 	 
final_transform, _ = multires_registration_interpolator(fixed_image2, moving_image2, initial_transform,interpolator)
moving_resampled = sitk.Resample(moving_image2, fixed_image2, final_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())

interact(display_images_with_alpha, image_z=(0,fixed_image.GetSize()[2] - 1), alpha=(0.0,1.0,0.05), fixed = fixed(fixed_image2), moving=fixed(moving_resampled))

Final metric value: -0.6832085594608565
Optimizer's stopping condition, GradientDescentOptimizerv4Template: Convergence checker passed at iteration 10.


interactive(children=(IntSlider(value=16, description='image_z', max=32), FloatSlider(value=0.5, description='…

<function __main__.display_images_with_alpha(image_z, alpha, fixed, moving)>

In [53]:
fixed_image2 = fixed_image
moving_image2 = moving_image
initial_transform = sitk.CenteredTransformInitializer(fixed_image2, 
                                                      moving_image2, 
                                                      sitk.Euler3DTransform(), 
                                                      sitk.CenteredTransformInitializerFilter.MOMENTS)
interpolator = sitk.sitkCosineWindowedSinc  	 
final_transform, _ = multires_registration_interpolator(fixed_image2, moving_image2, initial_transform,interpolator)
moving_resampled = sitk.Resample(moving_image2, fixed_image2, final_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())

interact(display_images_with_alpha, image_z=(0,fixed_image.GetSize()[2] - 1), alpha=(0.0,1.0,0.05), fixed = fixed(fixed_image2), moving=fixed(moving_resampled))

Final metric value: -0.6667945182898991
Optimizer's stopping condition, GradientDescentOptimizerv4Template: Convergence checker passed at iteration 10.


interactive(children=(IntSlider(value=16, description='image_z', max=32), FloatSlider(value=0.5, description='…

<function __main__.display_images_with_alpha(image_z, alpha, fixed, moving)>

In [54]:
fixed_image2 = fixed_image
moving_image2 = moving_image
initial_transform = sitk.CenteredTransformInitializer(fixed_image2, 
                                                      moving_image2, 
                                                      sitk.Euler3DTransform(), 
                                                      sitk.CenteredTransformInitializerFilter.MOMENTS)
interpolator = sitk.sitkLanczosWindowedSinc 	  	 
final_transform, _ = multires_registration_interpolator(fixed_image2, moving_image2, initial_transform,interpolator)
moving_resampled = sitk.Resample(moving_image2, fixed_image2, final_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())

interact(display_images_with_alpha, image_z=(0,fixed_image.GetSize()[2] - 1), alpha=(0.0,1.0,0.05), fixed = fixed(fixed_image2), moving=fixed(moving_resampled))

Final metric value: -0.6581270282125318
Optimizer's stopping condition, GradientDescentOptimizerv4Template: Convergence checker passed at iteration 9.


interactive(children=(IntSlider(value=16, description='image_z', max=32), FloatSlider(value=0.5, description='…

<function __main__.display_images_with_alpha(image_z, alpha, fixed, moving)>

In [55]:
fixed_image2 = fixed_image
moving_image2 = moving_image
initial_transform = sitk.CenteredTransformInitializer(fixed_image2, 
                                                      moving_image2, 
                                                      sitk.Euler3DTransform(), 
                                                      sitk.CenteredTransformInitializerFilter.MOMENTS)
interpolator = sitk.sitkBlackmanWindowedSinc  	  	 
final_transform, _ = multires_registration_interpolator(fixed_image2, moving_image2, initial_transform,interpolator)
moving_resampled = sitk.Resample(moving_image2, fixed_image2, final_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())

interact(display_images_with_alpha, image_z=(0,fixed_image.GetSize()[2] - 1), alpha=(0.0,1.0,0.05), fixed = fixed(fixed_image2), moving=fixed(moving_resampled))

Final metric value: -0.6469959510968252
Optimizer's stopping condition, GradientDescentOptimizerv4Template: Convergence checker passed at iteration 9.


interactive(children=(IntSlider(value=16, description='image_z', max=32), FloatSlider(value=0.5, description='…

<function __main__.display_images_with_alpha(image_z, alpha, fixed, moving)>

In [56]:
fixed_image2 = fixed_image
moving_image2 = moving_image
initial_transform = sitk.CenteredTransformInitializer(fixed_image6, 
                                                      moving_image6, 
                                                      sitk.VersorRigid3DTransform())
interpolator = sitk.sitkBlackmanWindowedSinc  	  	 
final_transform, _ = multires_registration_interpolator(fixed_image2, moving_image2, initial_transform,interpolator)
moving_resampled = sitk.Resample(moving_image2, fixed_image2, final_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())

interact(display_images_with_alpha, image_z=(0,fixed_image.GetSize()[2] - 1), alpha=(0.0,1.0,0.05), fixed = fixed(fixed_image2), moving=fixed(moving_resampled))

Final metric value: -0.7182477601325282
Optimizer's stopping condition, GradientDescentOptimizerv4Template: Convergence checker passed at iteration 10.


interactive(children=(IntSlider(value=16, description='image_z', max=32), FloatSlider(value=0.5, description='…

<function __main__.display_images_with_alpha(image_z, alpha, fixed, moving)>

In [57]:
fixed_image2 = fixed_image
moving_image2 = moving_image
initial_transform = sitk.CenteredTransformInitializer(fixed_image6, 
                                                      moving_image6, 
                                                      sitk.VersorRigid3DTransform())
interpolator = sitk.sitkLanczosWindowedSinc 	  	 
final_transform, _ = multires_registration_interpolator(fixed_image2, moving_image2, initial_transform,interpolator)
moving_resampled = sitk.Resample(moving_image2, fixed_image2, final_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())

interact(display_images_with_alpha, image_z=(0,fixed_image.GetSize()[2] - 1), alpha=(0.0,1.0,0.05), fixed = fixed(fixed_image2), moving=fixed(moving_resampled))

Final metric value: -0.6927225369533364
Optimizer's stopping condition, GradientDescentOptimizerv4Template: Convergence checker passed at iteration 10.


interactive(children=(IntSlider(value=16, description='image_z', max=32), FloatSlider(value=0.5, description='…

<function __main__.display_images_with_alpha(image_z, alpha, fixed, moving)>

In [58]:
fixed_image2 = fixed_image
moving_image2 = moving_image
initial_transform = sitk.CenteredTransformInitializer(fixed_image6, 
                                                      moving_image6, 
                                                      sitk.VersorRigid3DTransform())
interpolator = sitk.sitkCosineWindowedSinc 	
 	  	 
final_transform, _ = multires_registration_interpolator(fixed_image2, moving_image2, initial_transform,interpolator)
moving_resampled = sitk.Resample(moving_image2, fixed_image2, final_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())

interact(display_images_with_alpha, image_z=(0,fixed_image.GetSize()[2] - 1), alpha=(0.0,1.0,0.05), fixed = fixed(fixed_image2), moving=fixed(moving_resampled))

Final metric value: -0.6735712236108987
Optimizer's stopping condition, GradientDescentOptimizerv4Template: Convergence checker passed at iteration 10.


interactive(children=(IntSlider(value=16, description='image_z', max=32), FloatSlider(value=0.5, description='…

<function __main__.display_images_with_alpha(image_z, alpha, fixed, moving)>

In [59]:
fixed_image2 = fixed_image
moving_image2 = moving_image
initial_transform = sitk.CenteredTransformInitializer(fixed_image6, 
                                                      moving_image6, 
                                                      sitk.VersorRigid3DTransform())
interpolator = sitk.sitkHammingWindowedSinc 	
 	
 	  	 
final_transform, _ = multires_registration_interpolator(fixed_image2, moving_image2, initial_transform,interpolator)
moving_resampled = sitk.Resample(moving_image2, fixed_image2, final_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())

interact(display_images_with_alpha, image_z=(0,fixed_image.GetSize()[2] - 1), alpha=(0.0,1.0,0.05), fixed = fixed(fixed_image2), moving=fixed(moving_resampled))

Final metric value: -0.671799199693518
Optimizer's stopping condition, GradientDescentOptimizerv4Template: Convergence checker passed at iteration 9.


interactive(children=(IntSlider(value=16, description='image_z', max=32), FloatSlider(value=0.5, description='…

<function __main__.display_images_with_alpha(image_z, alpha, fixed, moving)>

In [60]:
fixed_image2 = fixed_image
moving_image2 = moving_image
initial_transform = sitk.CenteredTransformInitializer(fixed_image6, 
                                                      moving_image6, 
                                                      sitk.VersorRigid3DTransform())
interpolator = sitk.sitkGaussian  	
 	
 	  	 
final_transform, _ = multires_registration_interpolator(fixed_image2, moving_image2, initial_transform,interpolator)
moving_resampled = sitk.Resample(moving_image2, fixed_image2, final_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())

interact(display_images_with_alpha, image_z=(0,fixed_image.GetSize()[2] - 1), alpha=(0.0,1.0,0.05), fixed = fixed(fixed_image2), moving=fixed(moving_resampled))

Final metric value: -0.7609246109869774
Optimizer's stopping condition, GradientDescentOptimizerv4Template: Convergence checker passed at iteration 9.


interactive(children=(IntSlider(value=16, description='image_z', max=32), FloatSlider(value=0.5, description='…

<function __main__.display_images_with_alpha(image_z, alpha, fixed, moving)>

In [61]:
fixed_image2 = fixed_image
moving_image2 = moving_image
initial_transform = sitk.CenteredTransformInitializer(fixed_image6, 
                                                      moving_image6, 
                                                      sitk.VersorRigid3DTransform())
interpolator = sitk.sitkNearestNeighbor   	
 	
 	  	 
final_transform, _ = multires_registration_interpolator(fixed_image2, moving_image2, initial_transform,interpolator)
moving_resampled = sitk.Resample(moving_image2, fixed_image2, final_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())

interact(display_images_with_alpha, image_z=(0,fixed_image.GetSize()[2] - 1), alpha=(0.0,1.0,0.05), fixed = fixed(fixed_image2), moving=fixed(moving_resampled))

Final metric value: -0.6366183905110446
Optimizer's stopping condition, GradientDescentOptimizerv4Template: Convergence checker passed at iteration 9.


interactive(children=(IntSlider(value=16, description='image_z', max=32), FloatSlider(value=0.5, description='…

<function __main__.display_images_with_alpha(image_z, alpha, fixed, moving)>

In [62]:
fixed_image2 = fixed_image
moving_image2 = moving_image
transformDomainMeshSize = [8] * moving_image2.GetDimension()
initial_transform = sitk.TranslationTransform(fixed_image2.GetDimension())
interpolator = sitk.sitkGaussian  	
 	
 	  	 
final_transform, _ = multires_registration_interpolator(fixed_image2, moving_image2, initial_transform,interpolator)
moving_resampled = sitk.Resample(moving_image2, fixed_image2, final_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())

interact(display_images_with_alpha, image_z=(0,fixed_image.GetSize()[2] - 1), alpha=(0.0,1.0,0.05), fixed = fixed(fixed_image2), moving=fixed(moving_resampled))

Final metric value: -0.7521104600347911
Optimizer's stopping condition, GradientDescentOptimizerv4Template: Convergence checker passed at iteration 10.


interactive(children=(IntSlider(value=16, description='image_z', max=32), FloatSlider(value=0.5, description='…

<function __main__.display_images_with_alpha(image_z, alpha, fixed, moving)>

In [73]:
fixed_image2 = fixed_image
moving_image2 = moving_image
transformDomainMeshSize = [8] * moving_image2.GetDimension()
initial_transform = sitk.TranslationTransform(fixed_image2.GetDimension())
interpolator = sitk.sitkBlackmanWindowedSinc  	
 	
 	  	 
final_transform, _ = multires_registration_interpolator(fixed_image2, moving_image2, initial_transform,interpolator)
moving_resampled = sitk.Resample(moving_image2, fixed_image2, final_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())

interact(display_images_with_alpha, image_z=(0,fixed_image.GetSize()[2] - 1), alpha=(0.0,1.0,0.05), fixed = fixed(fixed_image2), moving=fixed(moving_resampled))

Final metric value: -0.6599915404310852
Optimizer's stopping condition, GradientDescentOptimizerv4Template: Convergence checker passed at iteration 10.


interactive(children=(IntSlider(value=16, description='image_z', max=32), FloatSlider(value=0.5, description='…

<function __main__.display_images_with_alpha(image_z, alpha, fixed, moving)>

## Setting optimizer
Optimizer will be changed for the best 2 initial transforms

In [69]:
fixed_image2 = fixed_image
moving_image2 = moving_image
initial_transform = sitk.CenteredTransformInitializer(fixed_image6, 
                                                      moving_image6, 
                                                      sitk.VersorRigid3DTransform())
registration_method = sitk.ImageRegistrationMethod()
registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
registration_method.SetMetricSamplingPercentage(0.01)
registration_method.SetInterpolator(sitk.sitkGaussian)
registration_method.SetOptimizerAsGradientDescentLineSearch(learningRate=1e-5, numberOfIterations=100, estimateLearningRate=registration_method.Once)
# registration_method.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=100, estimateLearningRate=registration_method.Once)
# registration_method.SetOptimizerScalesFromPhysicalShift() 
registration_method.SetInitialTransform(initial_transform, inPlace=False)
registration_method.SetShrinkFactorsPerLevel(shrinkFactors = [4,2,1])
registration_method.SetSmoothingSigmasPerLevel(smoothingSigmas = [2,1,0])
registration_method.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()
final_transform = registration_method.Execute(fixed_image2, moving_image2)
print('Final metric value: {0}'.format(registration_method.GetMetricValue()))
print('Optimizer\'s stopping condition, {0}'.format(registration_method.GetOptimizerStopConditionDescription()))
moving_resampled = sitk.Resample(moving_image2, fixed_image2, final_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())

interact(display_images_with_alpha, image_z=(0,fixed_image.GetSize()[2] - 1), alpha=(0.0,1.0,0.05), fixed = fixed(fixed_image2), moving=fixed(moving_resampled))

Final metric value: -0.635090001814894
Optimizer's stopping condition, GradientDescentLineSearchOptimizerv4Template: Convergence checker passed at iteration 31.


interactive(children=(IntSlider(value=16, description='image_z', max=32), FloatSlider(value=0.5, description='…

<function __main__.display_images_with_alpha(image_z, alpha, fixed, moving)>

In [72]:
fixed_image2 = fixed_image
moving_image2 = moving_image
initial_transform = sitk.CenteredTransformInitializer(fixed_image6, 
                                                      moving_image6, 
                                                      sitk.VersorRigid3DTransform())
registration_method = sitk.ImageRegistrationMethod()
registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
registration_method.SetMetricSamplingPercentage(0.01)
registration_method.SetInterpolator(sitk.sitkGaussian)
registration_method.SetOptimizerAsOnePlusOneEvolutionary()
# registration_method.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=100, estimateLearningRate=registration_method.Once)
# registration_method.SetOptimizerScalesFromPhysicalShift() 
registration_method.SetInitialTransform(initial_transform, inPlace=False)
registration_method.SetShrinkFactorsPerLevel(shrinkFactors = [4,2,1])
registration_method.SetSmoothingSigmasPerLevel(smoothingSigmas = [2,1,0])
registration_method.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()
final_transform = registration_method.Execute(fixed_image2, moving_image2)
print('Final metric value: {0}'.format(registration_method.GetMetricValue()))
print('Optimizer\'s stopping condition, {0}'.format(registration_method.GetOptimizerStopConditionDescription()))
moving_resampled = sitk.Resample(moving_image2, fixed_image2, final_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())

interact(display_images_with_alpha, image_z=(0,fixed_image.GetSize()[2] - 1), alpha=(0.0,1.0,0.05), fixed = fixed(fixed_image2), moving=fixed(moving_resampled))

Final metric value: -0.6289934223519654
Optimizer's stopping condition, OnePlusOneEvolutionaryOptimizerv4: Maximum number of iterations (100) exceeded. 


interactive(children=(IntSlider(value=16, description='image_z', max=32), FloatSlider(value=0.5, description='…

<function __main__.display_images_with_alpha(image_z, alpha, fixed, moving)>

In [74]:
fixed_image2 = fixed_image
moving_image2 = moving_image
initial_transform = sitk.CenteredTransformInitializer(fixed_image6, 
                                                      moving_image6, 
                                                      sitk.VersorRigid3DTransform())
registration_method = sitk.ImageRegistrationMethod()
registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
registration_method.SetMetricSamplingPercentage(0.01)
registration_method.SetInterpolator(sitk.sitkGaussian)
registration_method.SetOptimizerAsLBFGS2()
# registration_method.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=100, estimateLearningRate=registration_method.Once)
# registration_method.SetOptimizerScalesFromPhysicalShift() 
registration_method.SetInitialTransform(initial_transform, inPlace=False)
registration_method.SetShrinkFactorsPerLevel(shrinkFactors = [4,2,1])
registration_method.SetSmoothingSigmasPerLevel(smoothingSigmas = [2,1,0])
registration_method.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()
final_transform = registration_method.Execute(fixed_image2, moving_image2)
print('Final metric value: {0}'.format(registration_method.GetMetricValue()))
print('Optimizer\'s stopping condition, {0}'.format(registration_method.GetOptimizerStopConditionDescription()))
moving_resampled = sitk.Resample(moving_image2, fixed_image2, final_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())

interact(display_images_with_alpha, image_z=(0,fixed_image.GetSize()[2] - 1), alpha=(0.0,1.0,0.05), fixed = fixed(fixed_image2), moving=fixed(moving_resampled))

Final metric value: -0.8392910362147373
Optimizer's stopping condition, A rounding error occurred or line-search steps have an insufficient reduction


interactive(children=(IntSlider(value=16, description='image_z', max=32), FloatSlider(value=0.5, description='…

<function __main__.display_images_with_alpha(image_z, alpha, fixed, moving)>

In [75]:
fixed_image2 = fixed_image
moving_image2 = moving_image
transformDomainMeshSize = [8] * moving_image2.GetDimension()
initial_transform = sitk.TranslationTransform(fixed_image2.GetDimension())
registration_method = sitk.ImageRegistrationMethod()
registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
registration_method.SetMetricSamplingPercentage(0.01)
registration_method.SetInterpolator(sitk.sitkGaussian)
registration_method.SetOptimizerAsLBFGS2()
# registration_method.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=100, estimateLearningRate=registration_method.Once)
# registration_method.SetOptimizerScalesFromPhysicalShift() 
registration_method.SetInitialTransform(initial_transform, inPlace=False)
registration_method.SetShrinkFactorsPerLevel(shrinkFactors = [4,2,1])
registration_method.SetSmoothingSigmasPerLevel(smoothingSigmas = [2,1,0])
registration_method.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()
final_transform = registration_method.Execute(fixed_image2, moving_image2)
print('Final metric value: {0}'.format(registration_method.GetMetricValue()))
print('Optimizer\'s stopping condition, {0}'.format(registration_method.GetOptimizerStopConditionDescription()))
moving_resampled = sitk.Resample(moving_image2, fixed_image2, final_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())

interact(display_images_with_alpha, image_z=(0,fixed_image.GetSize()[2] - 1), alpha=(0.0,1.0,0.05), fixed = fixed(fixed_image2), moving=fixed(moving_resampled))

Final metric value: -0.8405566248597126
Optimizer's stopping condition, A rounding error occurred or line-search steps have an insufficient reduction


interactive(children=(IntSlider(value=16, description='image_z', max=32), FloatSlider(value=0.5, description='…

<function __main__.display_images_with_alpha(image_z, alpha, fixed, moving)>